After familiarizing yourself with the syntax of rgxlog, <br>
let's explore another example that highlights the power of this declarative language when integrated with Python. <br>
Consider three CSV files:

1. **parents.csv:** This file consists of two columns, indicating the parent-child relationship, with the left person being the parent of the right person.
2. **LastName.csv:** This file contains four rows, specifying the last names of individuals mentioned in the **parents.csv** file.
3. **hobbies.csv:** This file contains the hobbies of each person mentioned in **parents.csv**.

The task is to identify the family with the highest number of football players. <br>
It's important to note that you don't have access to individual last names, but you can deduce them by constructing family trees based on the information provided in **parents.csv**.

Take a couple of minutes to think a solution for this problem.

---

Now that you understand the complexity of deducing last names based on family trees when using Python, <br>
let us show you how easily we can do that by incorporating rgxlog into the equation.


In [ ]:
import rgxlog
import csv
from collections import defaultdict
from pprint import pprint

Let's start with importing the data of the CSV files into relations.

In [ ]:
from rgxlog import magic_session
magic_session.import_relation_from_csv("./sample_data/parents.csv", relation_name="parent", delimiter=",")
magic_session.import_relation_from_csv("./sample_data/LastName.csv", relation_name="LastName", delimiter=",")
magic_session.import_relation_from_csv("./sample_data/hobbies.csv", relation_name="Hobby", delimiter=",")

## Deducing Last Names Based on Family Trees with rgxlog

The following rgxlog code provides a simple yet powerful way to deduce the last name of each person based on the family tree information provided in the CSV files.

In [ ]:
%%rgxlog
Family(X,Y) <- LastName(X,Y)
Family(X,Y) <- Family(X,Z), parent(Y,Z)
Family(X,Y) <- Family(X,Z), parent(Z,Y)

This logic leverages the family tree structure encoded in the parents.csv file.<br>
Using only the 4 lines of data provided in LastName.csv we can deduce the last name to each individual appeared in the parents.csv file <br>
By recursively traversing parent-child relationships, the Family relation is established, <br>
allowing the deduced last names to be associated with each person. 

## Incorporating Python for Finding Families with Most Football Players

The beauty of spanner languages lies in the flexibility to seamlessly integrate different programming languages based on the requirements of the task at hand. <br>
In this scenario, after deducing the last names using rgxlog, we recognize the need to leverage Python's capabilities for further analysis. 

In [ ]:
# Lets start with sending the information extracted by rgxlog back to a csv file so python can take on from there
magic_session.send_commands_result_into_csv(commands="""
                                            FootballPlayersByFamily(X,Y) <- Family(X,Y), Hobby(Y,"Football")
                                            ?FootballPlayersByFamily(X,Y)
                                            """, 
                                            csv_file_name='./sample_data/FootballPlayersByFamily.csv',
                                            delimiter=','
                                            )

And now we can easily use minimal Python to use the extracted data and give us the answer for the question

In [ ]:
players_by_family = defaultdict(list)
with open('./sample_data/FootballPlayersByFamily.csv', 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        players_by_family[row[0]].append(row[-1])
        
sorted_families = sorted(players_by_family.keys(), key=lambda family : len(players_by_family[family]), reverse=True)
print(f"family with most Football players: {sorted_families[0]}")

family with most Football players: Smith
